In [ ]:
!pip install transformers
!pip install transformers_interpret

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/GPT

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/GPT


In [ ]:
import pandas as pd
### Number of attacked words
file_name= "DagPap22_roberta-base-openai-detector_predictions.csv"
n=15
df=pd.read_csv(file_name, index_col=[0])

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import ZeroShotClassificationExplainer, SequenceClassificationExplainer

tokenizer = AutoTokenizer.from_pretrained("roberta-base-openai-detector")

model = AutoModelForSequenceClassification.from_pretrained("roberta-base-openai-detector")

# zero_shot_explainer = ZeroShotClassificationExplainer(model, tokenizer)
class_explainer = SequenceClassificationExplainer(model, tokenizer)


Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw= stopwords.words('english')
import spacy
import requests
nlp = spacy.load("en_core_web_sm")
import re
from nltk.tokenize.treebank import TreebankWordDetokenizer as Detok
detokenizer = Detok()

def Sort(tuple): 
    # reverse = None (Sorts in Ascending order) 
    return(sorted(tuple, key = lambda a: -a[1]))  
def word_replace(word_attributions, n):
  wa= [(t[0], t[1], i) for i, t in enumerate(word_attributions)]
  wa= Sort(wa)
  wr= {t[0]:t[2] for t in wa[:n] if (t[0] !='') and (t[0] not in sw)}
  return wr

# Define a function to look up synonyms for a given word using the Datamuse API
def find_synonyms(word):
  url = "https://api.datamuse.com/words"
  params = {"rel_syn": word}
  response = requests.get(url, params=params)
  return [result["word"] for result in response.json()]

# Define a function to replace target words with appropriate synonyms in the context of the sentence
def replace_words_with_synonyms(text):
  doc = nlp(text)
  new_tokens = []
  replaced=""
  for i, token in enumerate(doc):
    if token.text in word_replacements:
      pos=word_replacements[token.text]
      if (pos-3 <= i < pos+3):
        context = token.sent.text
        synonyms = find_synonyms(token.text)
        if synonyms !=[]:
          new_tokens.append(synonyms[-1])
          replaced = replaced + "+"+ synonyms[-1]
        else:
          new_tokens.append(token.text)
      else:
        new_tokens.append(token.text)
    else:
      new_tokens.append(token.text)
  
  text = detokenizer.detokenize(new_tokens)
  text = re.sub('\s*,\s*', ', ', text)
  text = re.sub('\s*\.\s*', '. ', text)
  text = re.sub('\s*\?\s*', '? ', text)
  return text, replaced

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
df['text_attacked']= None
df['word_replacements']= None
df=df[df.source=='chatgpt']
df.reset_index(inplace=True)

In [ ]:
for i in range(len(df)):
  try:
    word_attributions = class_explainer(df.text[i])
    # html = class_explainer.visualize()
    word_replacements=word_replace(word_attributions,n)
    # df.loc[i,'word_replacements']= replaced
    df.loc[i,'text_attacked'], df.loc[i,'word_replacements'] = replace_words_with_synonyms(df.text[i])
    print(i)
  except:
    print("skipped: ", i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115


In [ ]:
df.to_csv(file_name.split(".csv")[0]+"_attacked_fixed.csv")

In [ ]:
df.text[0]

' It is no secret that children can be a great help around the house, and many believe that they should be required to help with household tasks as soon as they are able to do so. While there are some advantages to this, I disagree that it should be required of them.  For starters, children should not be expected to take on too much responsibility too soon. While it is important to teach them the value of hard work and to help them develop a sense of responsibility, they should not be expected to take on too much. Young children are still learning and developing, and they should be allowed to enjoy their childhood without feeling the pressure of too many obligations.  In addition, it is important to remember that children are not always able to do the same tasks as adults. Even if they are capable of helping out, they may not be able to do the same quality of work as an adult. Furthermore, tasks that require a certain level of skill or knowledge, such as cooking or cleaning, may be too

In [ ]:
df.text_attacked[0]

'It is no secret that children can be a great help around the planetary house, and many believe that they should be called upon to help with household tasks as soon as they are able to do so. While there are some advantages to this, I disagree that it should be required of them. For starters, children should not be expected to take on too much responsibility too before long. While it is important to teach them the value of hard work and to help them develop a sense of responsibility, they should not be expected to take on too much. schoolboyish children are still learning and developing, and they should be allowed to enjoy their childhood without feeling the pressure of too many obligations. In addition, it is important to remember that children are not always able to do the same tasks as adults. Even if they are capable of helping out, they may not be able to do the same upper-class of work as an adult. what is more, tasks that require a certain level of skill or knowledge, such as co

In [ ]:
word_attributions = class_explainer(df.text[0])
html = class_explainer.visualize()

In [ ]:
word_attributions = class_explainer(df.text_attacked[0])
html = class_explainer.visualize()